In [ ]:
pip install requests beautifulsoup4 psycopg2-binary



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import psycopg2

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       # Ganti dengan nama database Anda
    user="postgres",         # Ganti dengan username PostgreSQL Anda
    password="felixexe131105",     # Ganti dengan password PostgreSQL Anda
    host="localhost",         # Jika PostgreSQL berjalan di komputer lokal
    port="5432"                   # Port default PostgreSQL
)

cursor = connection.cursor()
print("Koneksi berhasil!")


Koneksi berhasil!


In [ ]:
import psycopg2
import requests
from bs4 import BeautifulSoup

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       
    user="postgres",             
    password="felixexe131105",   
    host="localhost",            # Ganti dengan host jika berbeda
    port="5432"                  # Port default PostgreSQL
)

cursor = connection.cursor()

# URL dari halaman e-commerce (contoh halaman produk)
url = "https://www.tokopedia.com/search?st=product&q=sepatu%20&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="

# Lakukan HTTP GET request
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

# Parse konten halaman menggunakan BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Cari elemen HTML yang berisi nama dan harga produk
produk = []
for item in soup.find_all("div", class_="_6+OpBPVGAgqnmycna+bWIw=="):  # Sesuaikan selector berdasarkan situs
    nama = item.find("span", class_="_0T8-iGxMpV6NEsYEhwkqEg==").get_text()  # Selector untuk nama produk
    
    # Cari harga produk
    harga_item = item.find_next("div", class_="_67d6E1xDKIzw+i2D2L0tjw== t4jWW3NandT5hvCFAiotYg==")
    if harga_item:
        harga = harga_item.get_text().strip()  # Ambil harga dan hilangkan spasi
        harga = int(harga.replace("Rp", "").replace(".", ""))  # Konversi harga ke integer (hapus "Rp" dan titik)
        produk.append((nama, harga))

# Masukkan data ke tabel ProdukEcommerce
for p in produk:
    try:
        # Query untuk memasukkan data ke dalam tabel
        query = "INSERT INTO ProdukEcommerce (nama_produk, harga) VALUES (%s, %s)"
        cursor.execute(query, (p[0], p[1]))  # Eksekusi query dengan data nama dan harga produk
        print(f"Data berhasil dimasukkan: {p[0]} | Harga: {p[1]}")
    except Exception as e:
        print(f"Error saat memasukkan da    ta: {e}")

# Commit perubahan ke database
connection.commit()

# Tutup koneksi
cursor.close()
connection.close()


Data berhasil dimasukkan: Sepatu Sneakers Wanita Puma Palermo Moda SurrealC Wns 39785402 | Harga: 143000
Data berhasil dimasukkan: Sepatu Sneakers Cassual Pria Wanita Classic Low Black Size  | Harga: 143000
Data berhasil dimasukkan: Sepatu Casual LiNing Soft Go | Harga: 49000
Data berhasil dimasukkan: Sepatu Loafers Wanita Terbaru - Sepatu Docmart Wanita Korea | Harga: 49000
Data berhasil dimasukkan: CANIKOREA Youngji Sepatu Sneakers Wanita Casual Korean Shoes 9252 | Harga: 550420
Data berhasil dimasukkan: Sepatu Compass Tribune White Blue | Harga: 550420
Data berhasil dimasukkan: Kanky Story Honjo - Sepatu Sneakers Casual Sport Sekolah Pria Dewasa | Harga: 550420
Data berhasil dimasukkan: PYOPP FLEDGE - Tabi-ku Barefoot Shoes - Black | Harga: 550420
Data berhasil dimasukkan: Kanky Klasik Story Kagayaku - Sepatu Sneakers Casual Sport Sekolah Pria Dewasa | Harga: 509150
Data berhasil dimasukkan: SEPATU SNEAKERS ORTUSEIGHT BRISBANE - ASH GREY | Harga: 509150


In [11]:
!pip install requests-html



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
pip install lxml[html_clean]


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import psycopg2
import time

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       
    user="postgres",             
    password="felixexe131105",   
    host="localhost",            
    port="5432"                  
)
cursor = connection.cursor()

# URL dari halaman e-commerce
url = "https://www.tokopedia.com/search?st=&q=yamaha%20&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="

# Setup Selenium WebDriver
service = Service('/path/to/chromedriver')  # Sesuaikan path ke chromedriver
driver = webdriver.Chrome(service=service)

# Akses halaman
driver.get(url)
time.sleep(5)  # Tunggu halaman selesai dimuat

# Parse halaman menggunakan BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Scraping data
produk = []
for item in soup.find_all("div", class_="css-1sn1xa2"):
    # Nama produk
    nama_tag = item.find("div", class_="css-1b6t4dn")
    nama_produk = nama_tag.get_text().strip() if nama_tag else None

    # URL produk
    link_tag = item.find("a", href=True)
    url_produk = link_tag["href"] if link_tag else None

    # Gambar produk
    img_tag = item.find("img", class_="success fade")
    img_url = img_tag["src"] if img_tag else None

    # Simpan data jika semua elemen ditemukan
    if nama_produk and url_produk and img_url:
        produk.append((nama_produk, url_produk, img_url))

# Masukkan data ke tabel CrawlingData
for p in produk:
    try:
        query = "INSERT INTO CrawlingData (nama_produk, url_produk, img_url) VALUES (%s, %s, %s)"
        cursor.execute(query, (p[0], p[1], p[2]))
        print(f"Data berhasil dimasukkan: {p[0]} | URL: {p[1]} | Gambar: {p[2]}")
    except Exception as e:
        print(f"Error saat memasukkan data: {e}")

# Commit dan tutup koneksi
connection.commit()
cursor.close()
connection.close()

print("Proses selesai.")


ModuleNotFoundError: No module named 'selenium'

In [4]:
import psycopg2
import requests
from bs4 import BeautifulSoup

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       
    user="postgres",             
    password="felixexe131105",   
    host="localhost",            
    port="5432"                  
)

cursor = connection.cursor()

# URL dari halaman e-commerce (contoh halaman produk)
url = "https://www.tokopedia.com/search?st=product&q=sepatu%20&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="

# Lakukan HTTP GET request
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

# Parse konten halaman menggunakan BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Cari elemen HTML yang berisi nama dan harga produk
produk = []
for item in soup.find_all("div", class_="css-5wh65g"):  # Selector lebih generik untuk kontainer produk
    try:
        # Cari nama produk
        nama_elem = item.find("span", class_="_0T8-iGxMpV6NEsYEhwkqEg==")
        nama = nama_elem.get_text(strip=True) if nama_elem else "Nama tidak ditemukan"

        # Cari harga produk
        harga_elem = item.find("div", class_="_67d6E1xDKIzw+i2D2L0tjw==")
        if harga_elem:
            harga = int(harga_elem.get_text(strip=True).replace("Rp", "").replace(".", ""))
        else:
            harga = 0

        # Tambahkan ke daftar produk
        produk.append((nama, harga))

    except Exception as e:
        print(f"Error saat memproses item: {e}")

# Masukkan data ke tabel ProdukEcommerce
for p in produk:
    try:
        # Query untuk memasukkan data ke dalam tabel
        query = "INSERT INTO ProdukEcommerce (nama_produk, harga) VALUES (%s, %s)"
        cursor.execute(query, (p[0], p[1]))
        print(f"Data berhasil dimasukkan: {p[0]} | Harga: {p[1]}")
    except Exception as e:
        print(f"Error saat memasukkan data: {e}")

# Commit perubahan ke database
connection.commit()

# Tutup koneksi
cursor.close()
connection.close()


Data berhasil dimasukkan: Sepatu Sneakers Wanita Puma Palermo Moda SurrealC Wns 39785402 | Harga: 1699000
Data berhasil dimasukkan: Sepatu Sneakers Cassual Pria Wanita Classic Low Black Size | Harga: 143000
Data berhasil dimasukkan: Sepatu Casual LiNing Soft Go | Harga: 890000
Data berhasil dimasukkan: Sepatu Loafers Wanita Terbaru - Sepatu Docmart Wanita Korea | Harga: 49000
Data berhasil dimasukkan: CANIKOREA Youngji Sepatu Sneakers Wanita Casual Korean Shoes 9252 | Harga: 135000
Data berhasil dimasukkan: Sepatu Compass Tribune White Blue | Harga: 568000
Data berhasil dimasukkan: Kanky Story Honjo - Sepatu Sneakers Casual Sport Sekolah Pria Dewasa | Harga: 318800
Data berhasil dimasukkan: PYOPP FLEDGE - Tabi-ku Barefoot Shoes - Black | Harga: 550420
Data berhasil dimasukkan: Kanky Klasik Story Kagayaku - Sepatu Sneakers Casual Sport Sekolah Pria Dewasa | Harga: 388800
Data berhasil dimasukkan: SEPATU SNEAKERS ORTUSEIGHT BRISBANE - ASH GREY | Harga: 509150
Data berhasil dimasukkan: Na

In [6]:
pip install selenium


   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB 48.5 kB/s eta 0:03:20
   ---------------------------------------- 0.0/9.7 MB 48.5 kB/s eta 0:03:20
   ---------------------------------------- 0.0/9.7 MB 48.5 kB/s eta 0:03:20
   -------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import psycopg2
import time

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",
    user="postgres",
    password="felixexe131105",
    host="localhost",
    port="5432"
)
cursor = connection.cursor()

# Konfigurasi Selenium
options = Options()
options.add_argument("--headless")  # Jalankan tanpa membuka browser
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Menggunakan WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Buka halaman e-commerce
url = "https://www.tokopedia.com/search?st=product&q=sepatu%20&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="
driver.get(url)
time.sleep(5)  # Tunggu halaman selesai di-load

# Scroll ke bawah untuk memuat lebih banyak data (opsional)
for _ in range(5):  # Sesuaikan jumlah scroll jika perlu
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(2)

# Ambil konten halaman
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

# Cari elemen produk
produk = []
for item in soup.find_all("div", class_="css-5wh65g"):  # Selector kontainer produk
    try:
        # Cari nama produk
        nama_elem = item.find("div", class_="css-1b6t4dn")
        nama = nama_elem.get_text(strip=True) if nama_elem else "Nama tidak ditemukan"

        # Cari URL gambar
        img_elem = item.find("img")
        img_url = img_elem["src"] if img_elem and "src" in img_elem.attrs else "URL gambar tidak ditemukan"

        # Tambahkan ke daftar produk
        produk.append((nama, img_url))

    except Exception as e:
        print(f"Error saat memproses item: {e}")

# Masukkan data ke tabel PostgreSQL
for p in produk:
    try:
        query = "INSERT INTO TabelCrawling (nama_produk, url_gambar) VALUES (%s, %s)"
        cursor.execute(query, (p[0], p[1]))
        print(f"Data berhasil dimasukkan: {p[0]} | URL Gambar: {p[1]}")
    except Exception as e:
        print(f"Error saat memasukkan data: {e}")

# Commit perubahan dan tutup koneksi
connection.commit()
cursor.close()
connection.close()

# Tutup browser
driver.quit()


In [15]:
pip install selenium webdriver-manager beautifulsoup4


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Konfigurasi Selenium
options = Options()
options.add_argument("--headless")  # Jalankan tanpa membuka browser
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Menggunakan WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Buka halaman e-commerce
url = "https://www.tokopedia.com/search?st=product&q=sepatu%20&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="
driver.get(url)
time.sleep(5)  # Tunggu halaman selesai di-load

# Scroll ke bawah untuk memuat lebih banyak data (opsional)
for _ in range(20):  # Coba tingkatkan jumlah scroll
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(3)  # Waktu tunggu lebih lama untuk memuat konten


# Ambil konten halaman
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")

# Cari semua elemen <img>
img_data = []
for img in soup.find_all("img"):
    try:
        # Ambil atribut src (URL gambar)
        img_url = img.get("src") or img.get("data-src") or "URL tidak ditemukan"
        # Ambil atribut alt (deskripsi gambar)
        img_alt = img.get("alt", "Deskripsi tidak ditemukan")
        img_data.append((img_alt, img_url))
    except Exception as e:
        print(f"Error saat memproses gambar: {e}")

# Simpan data ke file CSV
csv_file = "all_images_tokopedia.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Deskripsi Gambar (alt)", "URL Gambar"])  # Header
    writer.writerows(img_data)

print(f"Data berhasil disimpan ke {csv_file}")

# Tutup browser
driver.quit()


Data berhasil disimpan ke all_images_tokopedia.csv
